In [ ]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import json
import multiprocessing
import time

In [ ]:
# import file
tree = ET.parse('agios_atha.kml')
root = tree.getroot()

In [ ]:
# create empty array 
arr = np.empty((0,2))

# Το {http://www.opengis.net/kml/2.2} χρησιμοποιέιται για να ορίσεις την δομή και την διαδρομή του στοιχείου
# Ουσιαστικά δημιουργεί αντικείμενο επαναλήπτη που κάνει επαν΄αληψη για κάθε στοιχείο "Placemerk"
for i,placemark in enumerate(root.iter('{http://www.opengis.net/kml/2.2}Placemark')):
    # Εξαγωγή στοιχείων από το αρχείο
    # η συνάρτηση .find() χρησιμοποιείται για να πάρεις τις συντεταγμένες απο την διαδρομή point/coordinates
    coordinates = placemark.find('{http://www.opengis.net/kml/2.2}Point/{http://www.opengis.net/kml/2.2}coordinates').text
    
    # Προσθήκη x,y συντεταγμένων στον πίνακα
    # η συνάρτηση split επιστρέφει έναν πίνακα για όλα τα στοιχεία που έχουμε χωρίσει
    # χρησιμοποιούμε ξανά την split για το κενό και πάιρνουμε το τελευτείο στοιχείο
    x = float(coordinates.split(',')[0].split(" ")[-1])
    y = float(coordinates.split(',')[1])
    arr = np.append(arr,[[x,y]],axis=0)

In [ ]:
# Σχεδιάγραμμα στοιχείων
plt.scatter(arr[:,0],arr[:,1])
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Coordinate Plot')

# Adding annotations for each point
#for i, txt in enumerate(arr[:,0]):
#    plt.annotate(i+1, (arr[i][0], arr[i][1]), textcoords='offset points', xytext=(0, 10), ha='center')

plt.show()

In [ ]:
def path_lenght(array):
    # Υπλογίζουμε τις διαφο΄ρες μεταξύ συνεχόμενων σημείων 
    differences = np.diff(array, axis=0)  
    # Υπολογίζουμε ευκλείδιες αποστάσεις
    distances = np.linalg.norm(differences, axis=1)  
    # Αθροίζουμε τις αποστάσεις
    length = np.sum(distances)  
    # Προσθέτουμε τις αποστάσεις μεταξύ των πρώτου και τελευταίου σημείου 
    length += np.linalg.norm(array[-1] - array[0])  
    return length

def distance_matrix(points):
    # υπολογίζουμε ευκλείδιες αποστάσεις μεταξύ όλων των σημείων
    dist_mat = np.linalg.norm(points[:, np.newaxis] - points, axis=-1)
    return dist_mat


def inverse_distance_matrix(points):
    points_size = points.shape[0]
    
    # Calculate distance matrix using vectorized operations
    dist_mat = np.linalg.norm(points[:, np.newaxis] - points, axis=-1)
    
    # Δημιουργήστε έναν πίνακα ταυτότητας και υπολογίστε τις αντίστροφες αποστάσεις
    # np.where(condition,x,y)
    # np.eye is true at diag.
    # when cond is true, it assigns 0.0
    inv_dist_mat = np.where(np.eye(points_size, dtype=bool), 0.0, 1.0 / dist_mat)
    
    return inv_dist_mat

In [ ]:
dist_map = distance_matrix(arr)
sns.heatmap(dist_map)

In [ ]:
inv_dist_mat = inverse_distance_matrix(arr)
sns.heatmap(inv_dist_mat)

In [ ]:
class Ant:
    def __init__(self, n_locations):
        self.n_locations = n_locations
        self.position = np.random.choice(n_locations)
        
        self.places_visited = [self.position]
        self.places_left = list(np.arange(n_locations))
        self.places_left.remove(self.position)
        
        self.phero_graph = np.full((n_locations,n_locations),0.0)
        self.tour_cost = 0.0

    def ant_trip(self, g_phero_graph, dist_mat, inv_dist_mat, alpha=1, beta=1, Q=1):
        #range based loop to avoid unessecary list indexing
        for _ in range(len(self.places_left)):
            # vectorized operations
            allowed_weights = (g_phero_graph[self.position, self.places_left] ** alpha) * (inv_dist_mat[self.position, self.places_left] ** beta)
            allowed_weights_sum = np.sum(allowed_weights)
            travel_probas = allowed_weights / allowed_weights_sum

            next_destination = np.random.choice(self.places_left, p=travel_probas)

            self.tour_cost += dist_mat[self.position, next_destination]

            self.position = next_destination
            self.places_visited.append(next_destination)
            self.places_left.remove(next_destination)

        for i, j in zip(self.places_visited[:-1], self.places_visited[1:]):
            self.phero_graph[i, j] = Q / self.tour_cost

    def ant_flush(self):
        
        self.position = np.random.choice(self.n_locations)
        self.places_visited = [self.position]
        self.places_left = list(np.arange(self.n_locations))
        
        self.phero_graph = np.full((self.n_locations,self.n_locations),0.0)
        self.places_left.remove(self.position)
        self.tour_cost = 0.0


In [ ]:
def update_pheromones(g_phero_graph, ants, evapo_coef=0.05):
    # mult the evap_coef with current graph
    evaporation_component = (1 - evapo_coef) * g_phero_graph
    # sum all pheromone graphs of all the ants
    deposition_component = np.sum([ant.phero_graph for ant in ants], axis=0)
    # combine the evap component with new graph
    new_pheromones = evaporation_component + deposition_component
    # make sure they're below threshold(1e-08)
    new_pheromones = np.maximum(new_pheromones, 1e-08)
    return new_pheromones


In [ ]:
def aco(points, alpha, beta, evapo_coef, colony_size, num_iter):
    # Compute (once) the distance matrices
    dist_mat = distance_matrix(points)
    inv_dist_mat = inverse_distance_matrix(points)
    
    n_locations = points.shape[0]  # Total number of points
    ants = [Ant(n_locations) for _ in range(colony_size)]  # Ant colony
    
    # Determine initial pheromone value
    phero_init = (inv_dist_mat.mean())**(beta / alpha)
    g_phero_graph = np.full((n_locations, n_locations), phero_init)  # Pheromone matrix (arbitrary initialization)
    
    # Determine scaling coefficient "Q"
    [ant.ant_trip(g_phero_graph, dist_mat, inv_dist_mat, alpha, beta, Q=1) for ant in ants]
    best_ant_idx = np.argmin([ant.tour_cost for ant in ants])  # Find index of the best ant in this iteration
    Q = ants[best_ant_idx].tour_cost * phero_init / (0.1 * colony_size)
    print(Q)
    
    best_path_length = ants[best_ant_idx].tour_cost
    best_path = ants[best_ant_idx].places_visited
    
    monitor_costs = []
    unchanged_iters = 0 # counter for consecutive unchanced iterations
    
    for _ in tqdm(range(num_iter)):
        [ant.ant_trip(g_phero_graph, dist_mat, inv_dist_mat, alpha, beta, Q) for ant in ants]  # Ants perform their trips
        g_phero_graph = update_pheromones(g_phero_graph, ants, evapo_coef)  # Update pheromone levels
        
        iteration_winner_idx = np.argmin([ant.tour_cost for ant in ants])  # Find index of the best ant in this iteration
        best_path_iteration = ants[iteration_winner_idx].places_visited
        
        # update global best if better
        if best_path_length > ants[iteration_winner_idx].tour_cost:
            best_ant_idx = iteration_winner_idx
            best_path = best_path_iteration
            best_path_length = ants[iteration_winner_idx].tour_cost

            
        monitor_costs.append(best_path_length)
        
        [ant.ant_flush() for ant in ants]  # Reset ants for the next iteration
    
    return best_path, monitor_costs


In [ ]:
# ~3 times faster
monitor = []


best_path, monitor_costs = aco(points = arr, 
                               alpha = 1, 
                               beta = 1, 
                               evapo_coef = 0.2, 
                               colony_size = 75, 
                               num_iter = 100)





print(monitor_costs[-1])
print('')

plt.figure(figsize=(20,5));
plt.subplot(1,2,1);
plt.plot(arr[best_path,0],
         arr[best_path,1]);


plt.subplot(1,2,2);
plt.plot(np.arange(len(monitor_costs)),monitor_costs);

In [ ]:
# load results

# Load the JSON file
with open("save_file.json", "r") as file:
    data = json.load(file)

# Extract the 'monitor_costs' from the loaded data
monitor_costs = data['monitor_costs']
best_path = data['best_path']

print(monitor_costs[-1])
print('')

plt.figure(figsize=(20,5));
plt.subplot(1,2,1);
plt.plot(arr[best_path,0],
         arr[best_path,1]);


plt.subplot(1,2,2);
plt.plot(np.arange(len(monitor_costs)),monitor_costs);


In [ ]:
#save results


monitor_costs = [cost for cost in monitor_costs]
best_path = [int(point) for point in best_path]

# Save variables to a file
data = {'monitor_costs': monitor_costs, 'best_path': best_path}
with open('save_file.json', 'w') as file:
    json.dump(data, file)